In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1705105159375_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#importando bibliotecas
import os
import sys
import pytz
import numpy as np
import datetime
import time
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, udf, lpad, translate, to_timestamp, date_format
from datetime import datetime
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg, col, asc, desc, min, avg,max,sum,count,stddev, lit, when, concat

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#funcao freq
### FREQ FUNCTION
def Freq(pTabela,pColuna):

    qtd_total=pTabela.count()

    pTabela.registerTempTable("tab_input")
    frq = spark.sql(
            """
                select
                    {col},
                    count(*) as qtd_absoluto,
                    round(100*(count(*) / {tot}),2) as qtd_percentual
                from
                    tab_input
                group by
                    {col}
                order by
                    {col}
            """.format(col=pColuna, tot=qtd_total))

    qtd=frq.count()
    print('Quantidade de dominios',qtd)
    if qtd > 500:
        frq.show(1000,truncate=False)
        return "Dominio muito granular"

    else:
        frq.show(qtd,truncate=False)
        print("volumetria total:",qtd_total)
        return 'Freq da coluna ' + pColuna;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
agora = datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc = agora.strftime("%Y%m%d%H%M%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
spark = SparkSession.builder.appName("t_ssp_montly_file").getOrCreate()
sqlContext = SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
### MOD_PER_FILE
raw = "ingestion_ssp_montly_file"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
raw

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'ingestion_ssp_montly_file'

In [8]:
# Endereco Buckets (provisorio)
bucket_control =    "bkt-ssp/Files/05_Control"
bucket_ingestion =  "bkt-ssp/Files/01_IngestionZone"
bucket_raw =        "bkt-ssp/Files/02_RawZone"
bucket_trusted =    "bkt-ssp/Files/03_TrustedZone"
bucket_refined =    "bkt-ssp/Files/04_RefinedZone"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
### MOD_PER_FILE
output_trusted = "ingestion_ssp_montly_file"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
### MOD_PER_FILE
full_path_trusted = 's3://{bkt}/{file}'.format(bkt=bucket_raw,file=raw)
print(full_path_trusted)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-ssp/Files/02_RawZone/ingestion_ssp_montly_file

In [11]:
raw_00 = spark.read.format("parquet").load(full_path_trusted)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
raw_00.registerTempTable("raw_00")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
for coluna in raw_00.columns:
    print(coluna+',')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

id_municipio,
regiao_ssp,
homicidio_doloso,
numero_de_vitimas_em_homicidio_doloso,
homicidio_doloso_por_acidente_de_transito,
numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito,
homicidio_culposo_por_acidente_de_transito,
homicidio_culposo_outros,
tentativa_de_homicidio,
lesao_corporal_seguida_de_morte,
lesao_corporal_dolosa,
lesao_corporal_culposa_por_acidente_de_transito,
lesao_corporal_culposa_outras,
latrocinio,
numero_de_vitimas_em_latrocinio,
total_de_estupro,
estupro,
estupro_de_vulneravel,
total_de_roubo_outros,
roubo_outros,
roubo_de_veiculo,
roubo_a_banco,
roubo_de_carga,
furto_outros,
furto_de_veiculo,
ano,
mes,
ts_proc,
ts_proc_partition,

In [14]:
tabela = spark.sql(
"""
        SELECT
        
        -- PADRAO PARA TODAS AS TABELAS
        
            {pdthproc} as ts_proc,
            {pdthproc} as ts_proc_partition,
        
        -- CAMPOS DO ARQUIVO
            CAST(CONCAT(ano, lpad(mes, 2, '0')) AS STRING) as yyyymm,
            CAST(ano AS INTEGER) AS ano,
            CAST(mes AS INTEGER) AS mes,
            CAST(id_municipio AS STRING) AS id_municipio,
            CAST(regiao_ssp AS STRING) AS regiao_ssp,
        --
            CAST(homicidio_doloso AS INTEGER) AS homicidio_doloso,
            CAST(numero_de_vitimas_em_homicidio_doloso AS INTEGER) AS numero_de_vitimas_em_homicidio_doloso,
            CAST(homicidio_doloso_por_acidente_de_transito AS INTEGER) AS homicidio_doloso_por_acidente_de_transito,
            CAST(numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito AS INTEGER) AS numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito,
            CAST(homicidio_culposo_por_acidente_de_transito AS INTEGER) AS homicidio_culposo_por_acidente_de_transito,
            CAST(homicidio_culposo_outros AS INTEGER) AS homicidio_culposo_outros,
            CAST(tentativa_de_homicidio AS INTEGER) AS tentativa_de_homicidio,
            CAST(lesao_corporal_seguida_de_morte AS INTEGER) AS lesao_corporal_seguida_de_morte,
            CAST(lesao_corporal_dolosa AS INTEGER) AS lesao_corporal_dolosa,
            CAST(lesao_corporal_culposa_por_acidente_de_transito AS INTEGER) AS lesao_corporal_culposa_por_acidente_de_transito,
            CAST(lesao_corporal_culposa_outras AS INTEGER) AS lesao_corporal_culposa_outras,
            CAST(latrocinio AS INTEGER) AS latrocinio,
            CAST(numero_de_vitimas_em_latrocinio AS INTEGER) AS numero_de_vitimas_em_latrocinio,
            CAST(total_de_estupro AS INTEGER) AS total_de_estupro,
            CAST(estupro AS INTEGER) AS estupro,
            CAST(estupro_de_vulneravel AS INTEGER) AS estupro_de_vulneravel,
            CAST(total_de_roubo_outros AS INTEGER) AS total_de_roubo_outros,
            CAST(roubo_outros AS INTEGER) AS roubo_outros,
            CAST(roubo_de_veiculo AS INTEGER) AS roubo_de_veiculo,
            CAST(roubo_a_banco AS INTEGER) AS roubo_a_banco,
            CAST(roubo_de_carga AS INTEGER) AS roubo_de_carga,
            CAST(furto_outros AS INTEGER) AS furto_outros,
            CAST(furto_de_veiculo AS INTEGER) AS furto_de_veiculo
        FROM raw_00
"""
.format(pdthproc = dthproc))
tabela.registerTempTable("tabela")
tabela.cache()
tabela.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

153876

In [36]:
# Stack Colunas

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
manter = tabela.columns[:7]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
pivotar = tabela.columns[7:]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
manter

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ts_proc', 'ts_proc_partition', 'yyyymm', 'ano', 'mes', 'id_municipio', 'regiao_ssp']

In [40]:
pivotar

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['homicidio_doloso', 'numero_de_vitimas_em_homicidio_doloso', 'homicidio_doloso_por_acidente_de_transito', 'numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito', 'homicidio_culposo_por_acidente_de_transito', 'homicidio_culposo_outros', 'tentativa_de_homicidio', 'lesao_corporal_seguida_de_morte', 'lesao_corporal_dolosa', 'lesao_corporal_culposa_por_acidente_de_transito', 'lesao_corporal_culposa_outras', 'latrocinio', 'numero_de_vitimas_em_latrocinio', 'total_de_estupro', 'estupro', 'estupro_de_vulneravel', 'total_de_roubo_outros', 'roubo_outros', 'roubo_de_veiculo', 'roubo_a_banco', 'roubo_de_carga', 'furto_outros', 'furto_de_veiculo']

In [41]:
from pyspark.sql import functions as F

unpivotExp = ", ".join(f"'{col}', {col}" for col in pivotar)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
unpivotExp

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"'homicidio_doloso', homicidio_doloso, 'numero_de_vitimas_em_homicidio_doloso', numero_de_vitimas_em_homicidio_doloso, 'homicidio_doloso_por_acidente_de_transito', homicidio_doloso_por_acidente_de_transito, 'numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito', numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito, 'homicidio_culposo_por_acidente_de_transito', homicidio_culposo_por_acidente_de_transito, 'homicidio_culposo_outros', homicidio_culposo_outros, 'tentativa_de_homicidio', tentativa_de_homicidio, 'lesao_corporal_seguida_de_morte', lesao_corporal_seguida_de_morte, 'lesao_corporal_dolosa', lesao_corporal_dolosa, 'lesao_corporal_culposa_por_acidente_de_transito', lesao_corporal_culposa_por_acidente_de_transito, 'lesao_corporal_culposa_outras', lesao_corporal_culposa_outras, 'latrocinio', latrocinio, 'numero_de_vitimas_em_latrocinio', numero_de_vitimas_em_latrocinio, 'total_de_estupro', total_de_estupro, 'estupro', estupro, 'estupro_de_vulneravel', estupro_de_v

In [43]:
tabela_pivotada = tabela.selectExpr(*manter, f"stack({len(pivotar)}, {unpivotExp}) as (natureza, valor)"  )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
tabela_pivotada.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ts_proc', 'ts_proc_partition', 'yyyymm', 'ano', 'mes', 'id_municipio', 'regiao_ssp', 'natureza', 'valor']

In [50]:
# Ordem coluna
new_column_order = ['ts_proc', 'ts_proc_partition', 'yyyymm', 'ano', 'mes', 'id_municipio', 'regiao_ssp', 'natureza', 'valor']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
tabela_pivotada = tabela_pivotada.select(*new_column_order)
tabela_pivotada = tabela_pivotada.orderBy(asc('ts_proc'), asc('ts_proc_partition'), asc('yyyymm'), asc('Ano'), asc('mes'), asc('regiao_ssp'), asc('id_municipio'), asc('natureza'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
#Destino
path_trusted_complete = os.path.join('s3://',bucket_trusted,output_trusted)
print(path_trusted_complete)

tabela_pivotada.coalesce(1).write \
.partitionBy("ano", "yyyymm", "ts_proc", "ts_proc_partition") \
.parquet(path_trusted_complete, mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-ssp/Files/03_TrustedZone/ingestion_ssp_montly_file

In [ ]:
#End